In [1]:
import sys
sys.path.append('/home/iiyama/src/fastdla')
import numpy as np
import h5py
from qiskit.quantum_info import SparsePauliOp
from fastdla.dla import generate_dla, is_independent
from fastdla.sparse_pauli_vector import SparsePauliVector
from fastdla.ansatze.z2lgt_hva import z2lgt_hva_generators, z2lgt_gauss_projector, z2lgt_u1_projector, z2lgt_dense_projector

In [2]:
data_file = '/data/iiyama/z2lgt_hva_dla.h5'

## 2 Fermions

In [3]:
num_fermions = 2
# Determine the charge sector (symmetry subspace) to investigate
gauss_eigvals = [1, -1, 1, -1]
charge = 0

In [4]:
# Full HVA generators
generators_full = z2lgt_hva_generators(num_fermions)
generators_full

[SparsePauliVector(['IIIIIIIZ', 'IIIZIIII'], array([0.70710678+0.j, 0.70710678+0.j])),
 SparsePauliVector(['IIIIIZII', 'IZIIIIII'], array([0.70710678+0.j, 0.70710678+0.j])),
 SparsePauliVector(['IIIIIIXI', 'IIIIXIII', 'IIXIIIII', 'XIIIIIII'], array([0.5+0.j, 0.5+0.j, 0.5+0.j, 0.5+0.j])),
 SparsePauliVector(['IIIIIXZX', 'IIIIIYZY', 'IXZXIIII', 'IYZYIIII'], array([0.5+0.j, 0.5+0.j, 0.5+0.j, 0.5+0.j])),
 SparsePauliVector(['IIIXZXII', 'IIIYZYII', 'ZXIIIIIX', 'ZYIIIIIY'], array([0.5+0.j, 0.5+0.j, 0.5+0.j, 0.5+0.j]))]

In [5]:
# Compute the DLA of the full space
dla_full = generate_dla(generators_full, min_tasks=200, max_workers=54, verbosity=2)
print(f'DLA dimension is {len(dla_full)}')

Starting with 10 commutators..
Evaluating 10/10 commutators for independence
Adding 35 commutators; total 35
Current DLA dimension: 10
Evaluating 35/35 commutators for independence
Adding 280 commutators; total 280
Current DLA dimension: 26
Evaluating 255/280 commutators for independence
Adding 7178 commutators; total 7203
Current DLA dimension: 123
Evaluating 4528/7203 commutators for independence
Adding 40702 commutators; total 43377
Current DLA dimension: 311
Evaluating 37471/43377 commutators for independence
Current DLA dimension: 311
Evaluating 5906/5906 commutators for independence
Current DLA dimension: 311
DLA dimension is 311


In [6]:
# Projectors
gauss_projector = z2lgt_gauss_projector(gauss_eigvals)
u1_projector = z2lgt_u1_projector(num_fermions, charge)
symm_projector = gauss_projector @ u1_projector
# HVA generators and symmetry generators are simultaneously diagonalizable
# -> HVA generators can be projected onto the symmetry subspace by one-side application of the
# projectors. The resulting operators are the HVA generators in the subspace.
generators = [(op @ symm_projector).normalize() for op in generators_full]

In [7]:
# Check the dimensionality of the symmetry subspace
dense_projector = z2lgt_dense_projector(gauss_eigvals, charge)
print(f'Hilbert subspace dimension is {dense_projector.shape[0]}')

Hilbert subspace dimension is 12


In [8]:
# Refine the generators list in case some are linearly dependent in the subspace
generators_indep = [generators[0]]
for gen in generators[1:]:
    if is_independent(gen, generators_indep):
        generators_indep.append(gen)
print(f'{len(generators_indep)} generators are independent')

4 generators are independent


In [9]:
# Compute the DLA of the subspace
dla = generate_dla(generators_indep, min_tasks=200, max_workers=54, verbosity=2)
print(f'Subspace DLA dimension is {len(dla)}')

Starting with 6 commutators..
Evaluating 6/6 commutators for independence
Adding 30 commutators; total 30
Current DLA dimension: 9
Evaluating 30/30 commutators for independence
Adding 264 commutators; total 264
Current DLA dimension: 25
Evaluating 217/264 commutators for independence
Adding 1411 commutators; total 1458
Current DLA dimension: 59
Evaluating 342/1458 commutators for independence
Adding 305 commutators; total 1421
Current DLA dimension: 64
Evaluating 1421/1421 commutators for independence
Current DLA dimension: 64
Subspace DLA dimension is 64


In [10]:
with h5py.File(data_file, 'a') as out:
    if 'nf=2' not in out:
        nf_group = out.create_group('nf=2')
        for gname, oplist in [
            ('generators_full', generators_full),
            ('dla_full', dla_full),
            ('generators_symm0', generators_indep),
            ('dla_symm0', dla)
        ]:
            group = nf_group.create_group(gname)
            group.create_dataset('indices', data=np.concatenate([op.indices for op in oplist]))
            group.create_dataset('coeffs', data=np.concatenate([op.coeffs for op in oplist]))
            group.create_dataset('indptr', data=np.cumsum([0] + [op.num_terms for op in oplist]))

        nf_group.create_dataset('dla_symm0/gauss_eigvals', data=gauss_eigvals)
        nf_group.create_dataset('dla_symm0/charge', data=charge)

## 3 Fermions

In [11]:
num_fermions = 3
# Determine the charge sector (symmetry subspace) to investigate
gauss_eigvals = [1, -1, 1, -1, 1, -1]
charge = 0

In [12]:
# Full HVA generators
generators_full = z2lgt_hva_generators(num_fermions)
generators_full

[SparsePauliVector(['IIIIIIIIIIIZ', 'IIIIIIIZIIII', 'IIIZIIIIIIII'], array([0.57735027+0.j, 0.57735027+0.j, 0.57735027+0.j])),
 SparsePauliVector(['IIIIIIIIIZII', 'IIIIIZIIIIII', 'IZIIIIIIIIII'], array([0.57735027+0.j, 0.57735027+0.j, 0.57735027+0.j])),
 SparsePauliVector(['IIIIIIIIIIXI', 'IIIIIIIIXIII', 'IIIIIIXIIIII', 'IIIIXIIIIIII', 'IIXIIIIIIIII', 'XIIIIIIIIIII'], array([0.40824829+0.j, 0.40824829+0.j, 0.40824829+0.j, 0.40824829+0.j,
        0.40824829+0.j, 0.40824829+0.j])),
 SparsePauliVector(['IIIIIIIIIXZX', 'IIIIIIIIIYZY', 'IIIIIXZXIIII', 'IIIIIYZYIIII', 'IXZXIIIIIIII', 'IYZYIIIIIIII'], array([0.40824829+0.j, 0.40824829+0.j, 0.40824829+0.j, 0.40824829+0.j,
        0.40824829+0.j, 0.40824829+0.j])),
 SparsePauliVector(['IIIIIIIXZXII', 'IIIIIIIYZYII', 'IIIXZXIIIIII', 'IIIYZYIIIIII', 'ZXIIIIIIIIIX', 'ZYIIIIIIIIIY'], array([0.40824829+0.j, 0.40824829+0.j, 0.40824829+0.j, 0.40824829+0.j,
        0.40824829+0.j, 0.40824829+0.j]))]

In [13]:
# Compute the DLA of the full space
# dla_full = generate_dla(generators_full, min_tasks=100, max_workers=54, verbosity=3)
# print(f'DLA dimension is {len(dla_full)}')

In [14]:
# Projectors
gauss_projector = z2lgt_gauss_projector(gauss_eigvals)
u1_projector = z2lgt_u1_projector(num_fermions, charge)
symm_projector = gauss_projector @ u1_projector
# HVA generators and symmetry generators are simultaneously diagonalizable
# -> HVA generators can be projected onto the symmetry subspace by one-side application of the
# projectors. The resulting operators are the HVA generators in the subspace.
generators = [(op @ symm_projector).normalize() for op in generators_full]

In [15]:
# Check the dimensionality of the symmetry subspace
dense_projector = z2lgt_dense_projector(gauss_eigvals, charge)
print(f'Hilbert subspace dimension is {dense_projector.shape[0]}')

Hilbert subspace dimension is 40


In [16]:
# Refine the generators list in case some are linearly dependent in the subspace
generators_indep = [generators[0]]
for gen in generators[1:]:
    if is_independent(gen, generators_indep):
        generators_indep.append(gen)
print(f'{len(generators_indep)} generators are independent')

4 generators are independent


In [ ]:
# Compute the DLA of the subspace
dla = generate_dla(generators_indep, verbosity=2, min_tasks=200, max_workers=54)
print(f'Subspace DLA dimension is {len(dla)}')

Starting with 6 commutators..
Evaluating 6/6 commutators for independence
Adding 30 commutators; total 30
Current DLA dimension: 9
Evaluating 30/30 commutators for independence
Adding 264 commutators; total 264
Current DLA dimension: 25
Evaluating 126/264 commutators for independence
Adding 2703 commutators; total 2841
Current DLA dimension: 78
Evaluating 107/2841 commutators for independence
Adding 4500 commutators; total 7234
Current DLA dimension: 123
Evaluating 114/7234 commutators for independence
Adding 10833 commutators; total 17953
Current DLA dimension: 192
Evaluating 104/17953 commutators for independence
Adding 13290 commutators; total 31139
Current DLA dimension: 252
Evaluating 103/31139 commutators for independence
Adding 12330 commutators; total 43366
Current DLA dimension: 297
Evaluating 115/43366 commutators for independence
Adding 10659 commutators; total 53910
Current DLA dimension: 331
Evaluating 113/53910 commutators for independence
Adding 9646 commutators; total 6

In [ ]:
with h5py.File(data_file, 'a') as out:
    if 'nf=3' not in out:
        nf_group = out.create_group('nf=3')
        for gname, oplist in [
            ('generators_full', generators_full),
            # ('dla_full', dla_full),
            ('generators_symm0', generators_indep),
            ('dla_symm0', dla)
        ]:
            group = nf_group.create_group(gname)
            group.create_dataset('indices', data=np.concatenate([op.indices for op in oplist]))
            group.create_dataset('coeffs', data=np.concatenate([op.coeffs for op in oplist]))
            group.create_dataset('indptr', data=np.cumsum([0] + [op.num_terms for op in oplist]))

        nf_group.create_dataset('dla_symm0/gauss_eigvals', data=gauss_eigvals)
        nf_group.create_dataset('dla_symm0/charge', data=charge)